In [107]:
from math import pi, atan, sin, cos, radians

import cv2
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from tqdm import tqdm

from Generic import images, filedialogs
from ParticleTracking import dataframes, statistics

In [108]:
direc = "/media/data/Data/January2020/RecordFluctuatingInterface/Quick/first_frames"
files = filedialogs.get_files_directory(direc+'/*.png')
savename = direc + '/data.hdf5'

In [109]:
N = len(files)
ims = [images.load(f, 0) for f in tqdm(files, 'Loading images')]
circles = [images.find_circles(im, 27, 200, 7, 16, 16) for im in tqdm(ims, 'Finding circles')]


Finding circles: 100%|██████████| 50/50 [00:03<00:00, 14.23it/s]


In [110]:
data = dataframes.DataStore(savename, load=False)
for f, info in tqdm(enumerate(circles), 'Adding Circles'):
    data.add_tracking_data(f, info, ['x', 'y', 'r'])
    
calc = statistics.PropertyCalculator(data)
calc.order()

Adding Circles: 50it [00:00, 541.89it/s]


[########################################] | 100% Completed |  0.3s


In [111]:
def get_cgw(df):
    tree = spatial.cKDTree(df[['x', 'y']].values)
    dists, _ = tree.query(tree.data, 2)
    cgw = np.mean(dists[:, 1])
    return cgw

In [112]:
cgw = get_cgw(data.df.loc[0]) / 2

In [113]:
lattice_spacing = 5

In [114]:
# Get the lattice points
x = np.arange(0, max(data.df.x), lattice_spacing)
y = np.arange(0, max(data.df.y), lattice_spacing)
x, y = np.meshgrid(x, y)

In [115]:
def coarse_order_field(df, cgw, x, y, no_of_neighbours=20):
    """
    Calculate the coarse-grained field characterising local orientation order
    """

    order = df.order.values

    # Generate the lattice nodes to query
    # x, y = np.meshgrid(x, y)
    r = np.dstack((x, y))

    # Get the positions of all the particles
    particles = df[['x', 'y']].values

    # Generate the tree from the particles
    tree = spatial.cKDTree(particles)

    # Query the tree at all the lattice nodes to find the nearest n particles
    # Set n_jobs=-1 to use all cores
    dists, indices = tree.query(r, no_of_neighbours, n_jobs=-1)

    # Calculate all the coarse-grained delta functions (Katira ArXiv eqn 3
    cg_deltas = np.exp(-dists ** 2 / (2 * cgw ** 2)) / (2 * pi * cgw ** 2)

    # Multiply by the orders to get the summands
    summands = cg_deltas * order[indices]

    # Sum along axis 2 to calculate the field
    field = np.sum(summands, axis=2)

    return field, x, y

In [116]:
fields = []
for f in tqdm(range(N), 'Calculating fields'):
    field, x, y = coarse_order_field(data.df.loc[f], cgw, x, y)
    fields.append(field)

Calculating fields: 100%|██████████| 50/50 [00:07<00:00,  6.28it/s]


In [117]:
def get_field_threshold(fields, ls, im):
    # Draw a box around an always ordered region of the image to
    # calculate the phi_o
    fields = np.dstack(fields)
    line_selector = LineSelector(im)
    op1, op2 = line_selector.points
    phi_o = np.mean(
        fields[op1[1] // ls:op2[1] // ls, op1[0] // ls:op2[0] // ls, :])

    # Repeat for disordered
    line_selector = LineSelector(im)
    dp1, dp2 = line_selector.points
    phi_d = np.mean(
        fields[dp1[1] // ls:dp2[1] // ls, dp1[0] // ls:dp2[0] // ls, :])

    field_threshold = (phi_o + phi_d) / 2
    return field_threshold

In [118]:
class LineSelector:
    def __init__(self, im):
        cv2.namedWindow('line', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('line', 960, 540)
        cv2.setMouseCallback('line', self.record)
        self.points = []
        while True:
            cv2.imshow('line', im)
            key = cv2.waitKey(1) & 0xFF
            if len(self.points) == 2:
                break
        cv2.destroyAllWindows()

    def record(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.points.append([x, y])

In [119]:
field_threshold = get_field_threshold(fields, lattice_spacing, ims[0])

In [120]:
def find_contours(f, t):
    t_low = t - 0.02*t
    t_high = t + 0.02*5
    new_f = (f < t_high) * (f > t_low)
    new_f = np.uint8(new_f)
    contours = images.find_contours(new_f)
    contours = images.sort_contours(contours)
    return contours[-1]

In [195]:
contours = [find_contours(f, field_threshold) for f in fields]
# multiply contours by the spacing
contours = [c*lattice_spacing for c in contours]

In [122]:
ls = LineSelector(ims[0])

In [123]:
p1, p2 = ls.points
m = (p2[1] - p1[1])/(p2[0]-p1[0])
a = -atan(m)
c = np.array([i//2 for i in np.shape(ims[0])])[::-1]

In [124]:
def rotate_points(points, center, a):
    rot = np.array(((cos(a), -sin(a)), (sin(a), cos(a))))
    a1 = points - center
    a2 = rot@a1.T
    a3 = a2.T + center
    return a3
#     return center + rot@(points-center).T.squeeze().T

In [125]:
p1 = rotate_points(np.array(p1), c, a)
p2 = rotate_points(np.array(p2), c, a)
p1, p2

(array([231.12542145, 927.63568199]), array([1887.31532603,  927.63568199]))

In [196]:
%%time
contours = [rotate_points(contour.squeeze(), c, a) for contour in contours]

CPU times: user 1.6 ms, sys: 12.3 ms, total: 13.9 ms
Wall time: 5.2 ms


In [127]:
im = np.zeros((ims[0].shape[0]*3, ims[0].shape[1]*3))
im = cv2.polylines(im, [contours[0].astype('int32')], True, (255, 255, 255))
im = images.dilate(im, (3, 3))
# im = cv2.line(im, (int(p1[0]), int(p1[1])), (int(p2[0]), int(p2[1])), (255, 255, 255), 2)
images.plot(im)

In [128]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [129]:
h = p1[1]
xs = []
ys = []
xmin = int(p1[0])
xmax = int(p2[0])
for x in np.arange(xmin, xmax):
    crossings = np.argwhere(im[:, x] == 255)
    dists = crossings - h
    closest = np.argmin((crossings - h)**2)
    crossing = crossings[closest]
    ys.append(crossing)
    xs.append(x)

In [43]:
plt.plot(xs, ys)

In [197]:
def get_h(contour, im, p1, p2):
    xs = []
    ys = []
    xmin = int(p1[0])
    xmax = int(p2[0])
    h = p1[1]
    
    im = np.zeros((im.shape[0]*2, im.shape[1]*2))
    im = cv2.polylines(im, [contour.astype('int32')], True, (255, 255, 255))
    im = images.dilate(im, (3, 3))
    for x in np.arange(xmin, xmax):
        crossings = np.argwhere(im[:, x] == 255)
        dists = crossings - h
        closest = np.argmin((crossings-h)**2)
        crossing = crossings[closest]
        ys.append(crossing[0])
        xs.append(x)
    hs = [y-h for y in ys]
    return hs

In [198]:
%timeit
hs = [get_h(contour, ims[0], p1, p2) for contour in tqdm(contours)]

100%|██████████| 50/50 [00:07<00:00,  6.65it/s]


In [182]:
def get_fourier(hs, L):
    sp = [np.fft.fft(h) for h in hs]
    N = len(hs[0])
    freq = np.fft.fftfreq(N)
    
    y = np.stack(sp)
    y = np.mean(y, axis=0).squeeze()
    
    
    xplot = np.log(freq[1:N//2])
    yplot = np.log(L * np.abs(y[1:N // 2]) ** 2)
    
    p, cov = np.polyfit(xplot, yplot, 1, cov=True)
    p1 = np.poly1d(p)
      
    plt.plot(xplot, yplot)
    plt.plot(xplot, p1(xplot))
    plt.legend(['Data', 'Fit with gradient ${:.2f} \pm {:.2f}$'.format(p[0], cov[0][0]**0.5)])
    plt.xlabel('$k [$p$^{-1}]$')
    plt.ylabel('$ < |\delta h_k|^2 > L [$p$^3] $')

In [187]:
L = xmax - xmin
get_fourier(hs, L)

In [184]:
import seaborn as sns

In [185]:
sns.set()